# TP2- Image Quantization and Dithering

## Objective

- Understand the effect of color depth reduction (quantization) on an image.
- Implement and analyze the visual improvement provided by a common dithering algorithm, specifically Floyd-Steinberg Dithering.


In [ ]:
import cv2
import numpy as np


imagePath = "tp2-image.jpg"

## Part I - Color Quantization (No Dithering)

Goal: Reduce a 24-bit color image (True Color) to a lower bit depth (e.g., 3-bit or 8 unique colors) by simple truncation.

1. Load the Image: Load a standard color image (e.g., lena.jpg or a photo with a wide color palette) using cv2.imread().
2. Define New Palette: For a 3-bit color space, there are 23=8 possible colors. The colors are (0,0,0), (255,0,0), (0,255,0), (0,0,255), (255,255,0), (0,255,255), (255,0,255), and (255,255,255).
3. Quantize: For each pixel in the original image, map its 24-bit color to the nearest color in the 8-color palette. A simpler approach for this part is to divide the 8-bit color channels (0-255) to the nearest of 0 or 255 only (1-bit per channel → 8 colors total).

| Channel Value C (0-255) | Quantized Value C′ (0 or 255) |
| ----------------------- | ----------------------------- |
| C < 128                 | 0                             |
| C > 128                 | 255                           |

4. Display and Save: Display the original and the highly quantized image side-by-side using cv2.imshow().


In [ ]:
def simpleTruncation(imagePath):
    img = cv2.imread(imagePath)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            img[i, j, 0] = 255 if img[i, j, 0] > 127 else 0
            img[i, j, 1] = 255 if img[i, j, 1] > 127 else 0
            img[i, j, 2] = 255 if img[i, j, 2] > 127 else 0
    cv2.imshow("Image", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


simpleTruncation(imagePath)

# Part 2: Implementing Floyd-Steinberg Dithering

Goal: Apply the Floyd-Steinberg algorithm to distribute the quantization error to neighboring pixels, improving the perceived detail.
The Floyd-Steinberg algorithm is an error diffusion method that works by iterating over the pixels, quantizing a pixel, and then adding a weighted fraction of the resulting error to its neighbors that haven't been processed yet:

$$
\text{error} = \text{pixel}_\text{old} - \text{pixel}_\text{new}
$$

The error is distributed to the four unprocessed neighbors as follows:

| Neighbor             | Weight | Fraction |
| -------------------- | ------ | -------- |
| Right (+1,0)         | 7      | 167      |
| Bottom-Left (−1,+1)  | 3      | 163      |
| Bottom (0,+1)        | 5      | 165      |
| Bottom-Right (+1,+1) | 1      | 161      |

### Lab Steps for Part 2:

1. Convert and Prepare: Convert the input image to a floating-point type (e.g., np.float32) for accurate error calculations.
2. Iterate: Loop through every pixel (row y, column x) of the image.
3. Quantize: For the current pixel, calculate the new_pixel value (either 0 or 255 for each channel) using the same quantization rule from Part 1.
4. Calculate Error: Calculate the per-channel error between the original pixel value and the new quantized value.
5. Diffuse Error: Add the weighted fractions of the error to the appropriate neighbors (ensure you check the image boundaries before adding the error).
6. Display and Compare: Display the dithered image and compare it with the images from Part 1.


In [ ]:
def floydSteinberg(imagePath):
    # Read the image
    img = cv2.imread(imagePath)
    # Transform to float
    floatImage = np.array(img, np.float32)
    # Loop through each pixel
    for i in range(floatImage.shape[0]):
        for j in range(floatImage.shape[1]):
            # get current values
            r = floatImage[i, j, 0]
            g = floatImage[i, j, 1]
            b = floatImage[i, j, 2]
            # Quantize
            floatImage[i, j, 0] = 255 if floatImage[i, j, 0] > 127 else 0
            floatImage[i, j, 1] = 255 if floatImage[i, j, 1] > 127 else 0
            floatImage[i, j, 2] = 255 if floatImage[i, j, 2] > 127 else 0
            # Calculate error
            error_r = r - floatImage[i, j, 0]
            error_g = g - floatImage[i, j, 1]
            error_b = b - floatImage[i, j, 2]
            # Add the weighted error to the neighbors
            if j + 1 < floatImage.shape[1]:
                floatImage[i, j + 1, 0] += error_r * (7 / 167)
                floatImage[i, j + 1, 1] += error_g * (7 / 167)
                floatImage[i, j + 1, 2] += error_b * (7 / 167)

            if i + 1 < floatImage.shape[0] and j - 1 >= 0:
                floatImage[i + 1, j - 1, 0] += error_r * (3 / 163)
                floatImage[i + 1, j - 1, 1] += error_g * (3 / 163)
                floatImage[i + 1, j - 1, 2] += error_b * (3 / 163)

            if i + 1 < floatImage.shape[0]:
                floatImage[i + 1, j, 0] += error_r * (5 / 165)
                floatImage[i + 1, j, 1] += error_g * (5 / 165)
                floatImage[i + 1, j, 2] += error_b * (5 / 165)

            if i + 1 < floatImage.shape[0] and j + 1 < floatImage.shape[1]:
                floatImage[i + 1, j + 1, 0] += error_r * (1 / 161)
                floatImage[i + 1, j + 1, 1] += error_g * (1 / 161)
                floatImage[i + 1, j + 1, 2] += error_b * (1 / 161)

    cv2.imshow("New Dithered Image", floatImage)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


floydSteinberg(imagePath)